In [1]:
import pandas as pd
import chardet
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.subplots as sp

In [2]:
with open('data_science_job.csv', 'rb') as f:
    result = chardet.detect(f.read())
    
df = pd.read_csv('data_science_job.csv', encoding=result['encoding'])
df.head()

,Company,Job Title,Location,Job Type,Experience level,Salary,Requirment of the company,Facilities
0,SGS,Clinical Data Analyst,"Richardson, TX, United States",Full Time,Entry-level,48K+ *,"Computer Science,Data quality,Genetics,Mathema...",",,,,"
1,Ocorian,AML/CFT & Data Analyst,"Ebène, Mauritius",Full Time,Entry-level,48K+ *,"Agile,Data management,Finance,Security,,",",,,,"
2,Cricut,Machine Learning Engineer,"South Jordan, UT, United States",Full Time,NaN,90K+ *,"Agile,Architecture,AWS,Computer Science,Comput...","Career development,,,,"
3,Bosch Group,Application Developer & Data Analyst,"Nonantola, Italy",Full Time,Entry-level,48K+ *,"Engineering,Industrial,Oracle,Power BI,R,R&D",",,,,"
4,Publicis Groupe,Data Engineer Full time (Public Sector) USA,"Arlington, VA, United States",Full Time,Mid-level,108K+,"AWS,Azure,Computer Science,Consulting,Dataflow...","Flex hours,Flex vacation,Parental leave,Unlimi..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3198 entries, 0 to 3197
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Company                     3197 non-null   object
 1   Job Title                   3197 non-null   object
 2   Location                    3197 non-null   object
 3   Job Type                    3197 non-null   object
 4   Experience level            2962 non-null   object
 5   Salary                      3009 non-null   object
 6   Requirment of the company   3198 non-null   object
 7   Facilities                  3198 non-null   object
dtypes: object(8)
memory usage: 200.0+ KB


# Limpeza dos Dados

In [11]:
# Contar valores nulos
nulls = df.isnull().sum()

# Imprimir contagem e porcentagem de valores nulos por coluna
for column, count in nulls[nulls != 0].items():
    percentage = (count / df.shape[0]) * 100
    print(f"{column}: {count}  -->  {percentage:.2f}%")

Company: 1  -->  0.03%
Job Title: 1  -->  0.03%
Location: 1  -->  0.03%
Job Type: 1  -->  0.03%
Experience level: 4  -->  0.13%
Salary: 4  -->  0.13%


#### Tratando valores nulos nos salários com base no nível de experiência
Para lidar com os valores nulos na coluna "Salary", o código utiliza o nível de experiência como referência. Essa abordagem envolve preencher os valores ausentes com o salário médio correspondente ao respectivo nível de experiência.

In [5]:
# Converter a coluna "Salary" para valores numéricos e lidar com valores faltantes
df['Salary'] = df['Salary'].replace('[^\d.]', '', regex=True).replace('NaN', np.nan, regex=True).astype(float).apply(lambda x: x*1000)

# Calcular o salário médio para cada nível de experiência
mean_salary_by_level = df.groupby('Experience level')['Salary'].mean()

# Preencher os valores faltantes na coluna "Salary" com a média do respectivo nível de experiência
df['Salary'].fillna(df['Experience level'].map(mean_salary_by_level), inplace=True)

#### Tratando valores nulos nos níveis de experiência com base na média salarial de cada nível
Para lidar com os valores nulos na coluna "Experience level", o código utiliza a média salarial de cada nível. Essa abordagem permite preencher os valores ausentes com o nível de experiência que corresponde à média salarial mais próxima.

In [6]:
# Função para encontrar o nível de experiência mais próximo com base no salário
def find_nearest_level(row):
    salary = row['Salary']
    distances = abs(mean_salary_by_level - salary)
    closest_level = distances.idxmin()
    return closest_level

In [7]:
# Preencher os valores faltantes na coluna "Experience level" com o nível de experiência mais próximo com base no salário
df['Experience level'].fillna(df.apply(find_nearest_level, axis=1), inplace=True)

#### Mudando os valores vazios (,,) nas colunas "Requirements of the company" e "Facilities"

In [9]:
# Renomear a coluna "Requirment of the company" para "Requirements of the company
df.rename(columns={'Requirment of the company ': 'Requirements of the company'}, inplace=True)

# Remover valores vazios na coluna "Requirements of the company"
df['Requirements of the company'] = df['Requirements of the company'].replace(',,', '', regex=True)

# Substituir valores vazios por "No Requirements" na coluna "Requirements of the company"
for line in df['Requirements of the company']:
    if line == '':
        df['Requirements of the company'] = df['Requirements of the company'].replace('', 'No Requirements', regex=True)

In [10]:
# Remover valores vazios na coluna "Facilities"
df['Facilities'] = df['Facilities'].replace(',,', '', regex=True)

# Substituir valores vazios por "No Facilities" na coluna "Facilities" 
for line in df['Facilities']:
    if line == '':
        df['Facilities'] = df['Facilities'].replace('', 'No Facilities', regex=True)

#### Tratamento dos Dados Restantes

In [12]:
# Remove os valores nulos
df.dropna(inplace=True)

# Adiciona a coluna"City" com base na coluna "Location" 
df['City'] = df['Location'].apply(lambda x: x.split(',')[0])

df['City'] = df['City'].replace('Dublin 1', 'Dublin', regex=True)
 
# Adiciona a coluna  "Country" com base na coluna "Location"
df['Country'] = df['Location'].apply(lambda x: x.split(',')[-1])

# Reordena as colunas para corresponder à ordem desejada
df = df[['Company', 'Job Title', 'City', 'Country', 'Job Type', 'Experience level', 'Salary', 'Requirements of the company', 'Facilities']]

# Redefine o índice após remover as linhas
df.reset_index(drop=True, inplace=True)

df.head()

,Company,Job Title,City,Country,Job Type,Experience level,Salary,Requirements of the company,Facilities
0,SGS,Clinical Data Analyst,Richardson,United States,Full Time,Entry-level,48000.0,"Computer Science,Data quality,Genetics,Mathema...",No Facilities
1,Ocorian,AML/CFT & Data Analyst,Ebène,Mauritius,Full Time,Entry-level,48000.0,"Agile,Data management,Finance,Security",No Facilities
2,Cricut,Machine Learning Engineer,South Jordan,United States,Full Time,Executive-level,90000.0,"Agile,Architecture,AWS,Computer Science,Comput...",Career development
3,Bosch Group,Application Developer & Data Analyst,Nonantola,Italy,Full Time,Entry-level,48000.0,"Engineering,Industrial,Oracle,Power BI,R,R&D",No Facilities
4,Publicis Groupe,Data Engineer Full time (Public Sector) USA,Arlington,United States,Full Time,Mid-level,108000.0,"AWS,Azure,Computer Science,Consulting,Dataflow...","Flex hours,Flex vacation,Parental leave,Unlimi..."


# Análise Exploratória dos Dados

### 1) Quais são os cargos mais comuns entre as vagas de emprego?

In [14]:
# Agrupa os dados pelo título do cargo e conte o número de ocorrências
job_counts  = df['Job Title'].value_counts()

# Seleciona os 10 cargos mais comuns
top_jobs  = job_counts.head(10)

# Cria um DataFrame com os títulos dos cargos e suas contagens
data = pd.DataFrame({'Cargo': top_jobs.index, 'Contagem': top_jobs.values})

In [15]:
# Define cor de fundo e cor da fonte
background_color = 'rgb(0, 22, 33)'
font_color = 'rgb(240, 240, 240)'

# Gera um gráfico de barras utilizando o Plotly Express
fig = px.bar(data, x='Contagem', y='Cargo', orientation='h',
             labels={'Contagem': 'Contagem', 'Cargo': 'Cargo'},
             template='plotly_dark')

# Atualiza o layout com as cores desejadas e título
fig.update_layout(
    title="Cargos Mais Comuns entre as Vagas de Emprego",
    xaxis_title="Contagem",
    yaxis_title="Cargo",
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font_color=font_color,
)

# Define cores das barras com gradiente azul para verde
bar_colors = px.colors.sequential.haline_r[:len(data)]

# Atualiza cores das barras
fig.update_traces(marker=dict(color=bar_colors))

# Exibe o gráfico
fig.show()


Os cargos mais comuns entre as vagas de emprego são Data Engineer, Data Scientist e Data Analyst, de acordo com a contagem de ocorrências. O que faz muito sentido pelo fato de estarem abrangendo todos os niveis de experiência. Mas logo após esses 3, percebemos que a maiorias das vagas são para niveis de experiência mais altos, como Senior Data Scientist, Senior Data Engineer e Senior Data Analyst. Com isso podemos assumir que muitas dessas vagas que não especificam o nivel de experiência, sejam para niveis mais altos também. Ou seja, temos uma grande quantidade de vagas para profissionais mais experientes, e poucas vagas para profissionais mais juniores, abrindo hipóteses para a falta de oportunidades para profissionais que estão começando na área.

### 2) Qual é a distribuição dos tipos de emprego?

In [16]:
# Agrupa os dados pelo tipo de emprego e conte o número de ocorrências
job_type_counts = df['Job Type'].value_counts()

# Cria um DataFrame com os tipos de emprego e suas contagens
data = pd.DataFrame({'Tipo de Emprego': job_type_counts.index, 'Contagem': job_type_counts.values})

In [17]:
# Define cor de fundo e cor da fonte
background_color = 'rgb(0, 22, 33)'
font_color = 'rgb(240, 240, 240)'

# Define as cores desejadas
colors = ['rgb(0, 68,103)', 'rgb(0,150,112)', 'rgb(66,199, 70)']

# Gera um gráfico de pizza utilizando o Plotly Express
fig = px.pie(data, values='Contagem', names='Tipo de Emprego',
             labels={'Contagem': 'Contagem', 'Tipo de Emprego': 'Tipo de Emprego'},
             template='plotly_dark', color_discrete_sequence=colors)



# Atualiza o layout com as cores desejadas e título
fig.update_layout(
    title="Distribuição dos Tipos de Emprego",
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font_color=font_color,
)

# Exibe o gráfico
fig.show()

A distribuição dos tipos de emprego no dataset mostra que a maioria das vagas são para empregos em período integral (Full Time), enquanto existem apenas 2.35% das vagas são para estágio (Internship). Esses números indicam uma diferença significativa na quantidade de oportunidades disponíveis para estágios em comparação com empregos em tempo integral. Essa disparidade aumenta a hipotese de que há uma oferta limitada de vagas para iniciantes na área.

### 3) Quais os requisitos mais comuns entre as vagas de emprego de cada nivel de experiência?

In [18]:
new_df = df[['Experience level', 'Salary', 'Requirements of the company']].copy()

# Converte a coluna 'Requirements of the company' em colunas separadas para cada requisito usando get_dummies
requirements_dummies = new_df['Requirements of the company'].str.get_dummies(',')

# Junta as colunas convertidas ao DataFrame original
new_df = pd.concat([new_df, requirements_dummies], axis=1)

# Conta a frequência de cada requisito por nível de experiência
requirements_freq_by_experience = new_df.groupby('Experience level')[requirements_dummies.columns].sum()

# Cria uma lista vazia para armazenar os requisitos mais pedidos em cada nível de experiência
top_5_requirements_by_experience = []

# Para cada nível de experiência, seleciona os 5 requisitos mais pedidos
for experience_level, row in requirements_freq_by_experience.iterrows():
    top_5_requirements = row.nlargest(5).reset_index()
    top_5_requirements.columns = ['Requirement', 'Frequency']
    top_5_requirements['Experience level'] = experience_level
    top_5_requirements_by_experience.append(top_5_requirements)

# Concatena os DataFrames em uma única estrutura
top_5_requirements_by_experience = pd.concat(top_5_requirements_by_experience)

In [19]:
# Define cores manuais para as barras
bar_colors = ['rgb(0, 0, 24)', 'rgb(0, 10, 56)', 'rgb(0, 67, 100)', 'rgb(0, 147, 110)', 'rgb(62, 196, 67)']

# Cria subplots com base no número de níveis de experiência
num_rows = len(requirements_freq_by_experience)
fig = sp.make_subplots(rows=num_rows, cols=1, subplot_titles=requirements_freq_by_experience.index, vertical_spacing=0.1)

# Itera sobre cada nível de experiência e crie gráficos de barras
for i, (experience_level, row) in enumerate(requirements_freq_by_experience.iterrows()):
    top_5_requirements = row.nlargest(5)
    fig.add_trace(go.Bar(x=top_5_requirements.index, y=top_5_requirements.values, marker=dict(color=bar_colors), name='Requisitos'),
                  row=i+1, col=1)

# Atualiza os títulos dos eixos e o layout geral
fig.update_xaxes(title_text='Requisitos', row=num_rows, col=1)
fig.update_yaxes(title_text='Frequência', row=int(num_rows/2), col=1)
fig.update_layout(height=1000, title="Top 5 Requisitos Mais Pedidos em Cada Nível de Experiência",
                  template='plotly_dark', )

fig.update_traces(showlegend=False)

# Exibe o gráfico
fig.show()

Analisando os requisitos mais comuns entre as vagas de emprego de cada nivel de experiência, podemos perceber que a análise de dados é mais comum nos níveis iniciais, incluindo requisitos como Excel e Data analysis. À medida que o nível de experiência aumenta, o foco muda para construção de algoritmos e infraestrutura e os requisitos se tornam mais específicos, como AWS, Architecture e Big Data. Esses requisitos mais específicos podem ser um indicador de que as vagas de emprego para níveis mais altos exigem mais conhecimento e experiência.

### 4) Quais os requisitos das empresas possuem a maior média de salario oferecido?

In [20]:
requirements = []
means_salaries = []

# Calcule a média de salário para cada requisito com pelo menos três vagas
for column in requirements_dummies.columns:
    vacancy_count = new_df[new_df[column] == 1].shape[0]
    if vacancy_count >= 3:
        mean_salary = new_df.loc[new_df[column] == 1, 'Salary'].mean()
        requirements.append(column)
        means_salaries.append(mean_salary)

# Crie um DataFrame para armazenar as médias de salários por requisito
mean_salary_by_requirement = pd.DataFrame({'Requirements': requirements, 'Mean Salary': means_salaries})

# Ordenar o DataFrame em ordem decrescente pela média de salário
mean_salary_by_requirement = mean_salary_by_requirement.sort_values(by='Mean Salary', ascending=False)

# Selecionar os 10 primeiros requisitos com os maiores salários médios
top_10_requirements = mean_salary_by_requirement.head(10)

In [21]:
# Definir cor de fundo e cor da fonte
background_color = 'rgb(0, 22, 33)'
font_color = 'rgb(240, 240, 240)'

# Gerar um gráfico de barras utilizando o Plotly Express
fig = px.bar(top_10_requirements, x='Mean Salary', y='Requirements', orientation='h',
             labels={'Mean Salary': 'Média de Salário', 'Requirements': 'Requisito'},
             template='plotly_dark')

# Atualizar o layout com as cores desejadas e título
fig.update_layout(
    title="Top 10 Requisitos com Maiores Salários Médios",
    xaxis_title="Média de Salário",
    yaxis_title="Requisito",
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font_color=font_color,
)

# Definir cores das barras com gradiente azul para verde
bar_colors = px.colors.sequential.haline_r[:10]

# Atualizar cores das barras
fig.update_traces(marker=dict(color=bar_colors))

fig.show()

Ao analisar os requisitos das empresas com a maior média de salário oferecido, podemos observar que esses requisitos são bastante específicos e não tão comuns de serem encontrados no dia a dia. Esses requisitos refletem a demanda por habilidades e conhecimentos especializados, o que pode resultar em salários mais elevados. É importante destacar que por esses requisitos estarem relacionados a áreas como inteligência artificial, aprendizado de máquina e ciência de dados, a expertise nesses domínios é altamente valorizada pelas empresas.

### 5) Quais as cidades e as empresas que oferecem os melhores salários?

In [22]:
# Agrupa os dados por cidade e empresa e calcular o salário médio para cada grupo
avg_salary_by_city = df.groupby('City')['Salary'].mean()
avg_salary_by_company = df.groupby('Company')['Salary'].mean()

# Filtra as cidades e empresas que possuem pelo menos 3 vagas de emprego
cities_with_min_3_jobs = df['City'].value_counts()[df['City'].value_counts() >= 3].index
companies_with_min_3_jobs = df['Company'].value_counts()[df['Company'].value_counts() >= 3].index

# Calcula o salário médio para as cidades e empresas com pelo menos 3 vagas de emprego
avg_salary_by_city = avg_salary_by_city[cities_with_min_3_jobs]
avg_salary_by_company = avg_salary_by_company[companies_with_min_3_jobs]

# Ordena as cidades e empresas com base no salário médio em ordem decrescente
top_cities = avg_salary_by_city.sort_values(ascending=False).head(5)
top_companies = avg_salary_by_company.sort_values(ascending=False).head(5)

In [23]:
# Cria subplots com 2 linhas e 1 coluna
fig = sp.make_subplots(rows=2, cols=1, subplot_titles=("Cidades com os Maiores Salários", "Empresas com os Maiores Salários"), vertical_spacing=0.1)

# Cria gráficos de barras para as principais cidades e empresas
fig.add_trace(go.Bar(x=top_cities.index, y=top_cities.values, marker=dict(color=bar_colors), name='Cities'), row=1, col=1)
fig.add_trace(go.Bar(x=top_companies.index, y=top_companies.values, marker=dict(color=bar_colors), name='Companies'), row=2, col=1)

# Atualiza layout com as cores e títulos desejados
fig.update_layout(
    title="Cidades e Empresas com os Maiores Salário",
    yaxis=dict(title="Salário Médio"),
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font_color=font_color,
    template='plotly_dark',
    height=900  
)

fig.update_traces(showlegend=False)

fig.show()

Podemos observar que a maioria das cidades com os salários mais altos, com exceção de Brooklyn, está localizada na costa oeste dos Estados Unidos, assim como as empresas também estão concentradas nos estados de Washington e Califórnia. Isso sugere que essa região geográfica pode oferecer oportunidades com salários mais altos no setor

### 6) Qual a distribuição de salários por nível de experiência?

In [24]:
# Define cor de fundo e cor da fonte
background_color = 'rgb(0, 22, 33)'
font_color = 'rgb(240, 240, 240)'

# Gera um gráfico de boxplot utilizando o Plotly Express
fig = px.box(df, x='Experience level', y='Salary',
             labels={'Experience level': 'Nível de Experiência', 'Salary': 'Salário'},
             template='plotly_dark')

# Atualiza o layout com as cores desejadas e título
fig.update_layout(
    title="Distribuição de Salários por Nível de Experiência",
    xaxis_title="Nível de Experiência",
    yaxis_title="Salário",
    plot_bgcolor=background_color,
    paper_bgcolor=background_color,
    font_color=font_color,
)

# Atualiza cores das caixas
fig.update_traces(marker=dict(color='darkcyan'))

# Exibe o gráfico
fig.show()

Com base nos dados fornecidos, observa-se que há diferenças significativas nos salários entre os níveis de experiência. O nível "Senior-level" apresenta o salário mais alto. Por outro lado, o nível "Executive-level" tem uma média de salário menor em comparação ao nível "Senior-level". Essa diferença indica que, em média, profissionais com nível "Senior" têm salários mais elevados do que aqueles com nível "Executive". Além disso, o nível "Entry-level" tem o salário mais baixo, o que sugere que os profissionais com menos experiência recebem salários mais baixos.
